Use a pretrained network to extract features
    + Load the pretrained model.
    + Create numpy arrays and fill them by running data through  pretrained model.
    + 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Load pretrained model

In [2]:
from keras.applications import VGG16
from keras.models import Sequential
from keras import layers

c:\users\a00439512\appdata\local\continuum\anaconda3\envs\demo\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
conv_base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))

model = Sequential()

model.add(conv_base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [5]:
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 30


In [6]:
#print(type(model.trainable_weights))

conv_base_model.trainable = False

ll = model.trainable_weights
print(ll[0])
print(ll[1])
print(ll[2])
print(ll[3])

<tf.Variable 'dense_1/kernel:0' shape=(8192, 256) dtype=float32_ref>
<tf.Variable 'dense_1/bias:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'dense_2/kernel:0' shape=(256, 1) dtype=float32_ref>
<tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32_ref>


In [7]:
conv_base_model.trainable = False

### SetUp Data sources

In [8]:
import os
import numpy as np

In [9]:
pwd_ = os.getcwd()
pwd_

'D:\\Github\\DeepNets\\Keras\\Keras_from_scratch\\Vision'

In [10]:
path =  os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(pwd_)))))
print(path)

D:\


In [11]:
dest_data_folder = os.path.join(os.path.join(path, "Warehouse"), "cats_and_dogs")

train_dir = os.path.join(dest_data_folder, "train")
cat_train_dir = os.path.join(train_dir, "cat")
dog_train_dir = os.path.join(train_dir, "dog")


test_dir = os.path.join(dest_data_folder, "test")
cat_test_dir = os.path.join(test_dir, "cat")
dog_test_dir = os.path.join(test_dir, "dog")


validation_dir = os.path.join(dest_data_folder, "validation")
cat_validation_dir = os.path.join(validation_dir, "cat")
dog_validation_dir = os.path.join(validation_dir, "dog")


### Extracting features using the pretrained convolutional base

In [12]:
from keras.preprocessing.image import ImageDataGenerator

In [13]:


from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 4000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
num_of_epochs = 1 # ideally this sould be 40 or above

from keras import optimizers


model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=num_of_epochs,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

## this is painfully slow

Epoch 1/1


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
    
plt.show()